Kaggle URL: [Yolo11s Train Augmentation](https://www.kaggle.com/code/phatle1578/yolo11s-train-augmentation)

In [ ]:
# ====================================================
# 1️⃣ CÀI ĐẶT MÔI TRƯỜNG (ĐÃ FIX RAY + TENSORBOARD)
# ====================================================
!pip uninstall -y ray ray[default] ray[tune] >/dev/null 2>&1 || true
!pip -q install ultralytics==8.3.27 opencv-contrib-python==4.10.0.84 tqdm==4.67.1 torch==2.1.2 torchvision==0.16.2 open_clip_torch==2.24.0 timm==0.9.12 # (P) add last library
# 🔧 FIX lỗi TensorBoard / protobuf conflict
!pip install -q "numpy<2.0"
!pip install -q protobuf==3.20.3 tensorboard==2.14.0
!pip install -q filterpy

In [ ]:
# ====================================================
# 2️⃣ CHUẨN BỊ DỮ LIỆU
# ====================================================
import os, re, json, shutil, random, cv2
import numpy as np
from tqdm import tqdm

In [ ]:
AUG_DATA = "/kaggle/input/yolo11s-augmentation-dataset-creation/zaic_yolo/yolo_dataset"
DATA_YAML = "/kaggle/input/yolo11s-augmentation-dataset-creation/zaic_yolo/data.yaml"
WORK_DIR = "/kaggle/working"

In [ ]:
!cp $DATA_YAML /kaggle/working/
DATA_YAML = "/kaggle/working/data.yaml"
print(DATA_YAML)

In [ ]:
with open(DATA_YAML, 'r') as f:
    data_config = yaml.safe_load(f)

# 1. Cập nhật giá trị trong từ điển bằng các biến đường dẫn bạn đã khai báo
# Sử dụng IMG_PATH vì YOLO cần đường dẫn đến thư mục chứa ảnh
data_config['path'] = AUG_DATA

# 2. Ghi đè lại nội dung đã sửa vào file data.yaml
with open(DATA_YAML, 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)

In [ ]:
# ====================================================
# 4️⃣ TRAIN YOLOv8 (FIX CALLBACK)
# ====================================================
from ultralytics import YOLO
from ultralytics.utils import callbacks
if "raytune" in callbacks.default_callbacks:
    del callbacks.default_callbacks["raytune"]

In [ ]:
model = YOLO("yolo11s.pt")
results = model.train(
    data=DATA_YAML, epochs=40, imgsz=960, batch=16, device=0,
    workers=2, patience=20, optimizer="auto", lr0=0.002,
    degrees=5.0, translate=0.05, scale=0.5, shear=0.0,
    perspective=0.0008, fliplr=0.5, mosaic=1.0, mixup=0.15,
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
    project="/kaggle/working/yolo_train",
    name="yolo11s_object",
)